#### VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes

###### Author: Nishchal Gaba
###### Created on: January 12, 2022

In [13]:
# Import packages
import os
import time
import sys
import random
from datetime import datetime, timedelta

import json
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

#### Utility Methods

In [14]:
# Used for Coin Gecko
token_names = {"DAI"  : 'dai',
            "GUSD"  : 'gemini-dollar',
            # "SUSD"  : 'susd',
            "TUSD"  : 'true-usd',
               
               
            "USDC"  : 'usd-coin',
            # "USDP"  : 'usdp',
            "USDT"  : "tether",
            "BAL"   : "balancer",
            "WETH"   : "ethereum",
            "LINK"  : "chainlink",
            "MKR"   : "maker",
            "RAI"   : "rai",
            "UNI"   : "uniswap",
            "WBTC"  : "wrapped-bitcoin",
            "XSUSHI": "xsushi",
            "YFI"   : "yearn-finance",
            "BUSD"  : "binance-usd",
            "FEI"   : "fei-usd",
            "FRAX"  : "frax",
            "AAVE"  : "aave",
            "AMPL"  : "ampleforth",
            "BAT"   : "basic-attention-token",
            "CRV"   : "curve-dao-token",
            "DPI"   : "defipulse-index",
            # # "ENJ"   : "enjin-coin",
            "KNC"   : "kyber-network-crystal",
            "MANA"  : "decentraland",
            # "REN"   : "ren",
            "RENFIL": "renfil",
            # "SNX"   :"synthetix-network-token",
            "ZRX"   : "0x"
            }

In [15]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)
    

In [17]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)


In [18]:
def change():
    query = '''
                {
                reserves
                (where:{
                    symbol:"WBTC"
                    })
                {
                    id
                    symbol
                    name
                    usageAsCollateralEnabled
                        borrowingEnabled
                    baseLTVasCollateral
                    totalDeposits
                    totalLiquidity
                    totalATokenSupply
                    reserveFactor
                    
                }
                }
            '''

    V2_sample_transport       = RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
        verify=True,
        retries=5)
    V2_client                 = Client(transport=V2_sample_transport)
    response                  = V2_client.execute(gql(query))

    names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        # "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        # "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        # "REN"   : "ren",
        "RENFIL": "renfil",
        # "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }

    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')


    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375
    LIMIT     = 5
    WBTC      = []

    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt

    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]
        
        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
        df['timestamp']  = tt

    df['timestamp'] = tt

    for asset in list(names.keys()):#[:LIMIT]:
        
        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change
        
        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

        
    return df

In [19]:
# # AAVE V2 Protocol tokens as provided in the sheet
# # Mapping from normal symbols to CoinGecko symbols (token_names)
# AAVE_V2_PROTOCOL_TOKENS = {
#     "BTC":"WBTC",
#     "ETH":"WETH",
#     "DOGE":"",
#     "MIST":"",
#     "USDC":"USDC",
#     "USDT":"USDT",
#     "GUSD":"GUSD",
#     "SUSD":"",
#     "TUSD":"TUSD",
#     "USDP":"USDP",
#     "BAL":"BAL",
#     "LINK":"LINK",
#     "MKR":"MKR",
#     "RAI":"RAI",
#     "UNI":"UNI",
#     "SUSHI":"",
#     "XSUSHI":"XSUSHI",
#     "YFI":"YFI",
#     "BUSD":"BUSD",
#     "FEI":"FEI",
#     "FRAX":"FRAX",
#     "COMP":"COMP",
#     "AAVE":"AAVE",
#     "AMPL":"AMPL",
#     "BAT":"BAT",
#     "CRV":"CRV",
#     "DPI":"DPI",
#     "ENJ":"ENJ",
#     "KNC":"KNC",
#     "MANA":"MANA",
#     "REN":"",
#     "RENFIL":"",
#     "SNX":"SNX",
#     "ZRX":"ZRX",
#     "REP":"",
#     "SAI":"",
#     "DAI":"DAI", 
# }

#### Getting the AAVEV2 assets via Subgraph GQL


In [20]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [21]:
query = '''
                {
                reserves
                (where:{
                    symbol:"WBTC"
                    })
                {
                    id
                    symbol
                    name
                    usageAsCollateralEnabled
                        borrowingEnabled
                    baseLTVasCollateral
                    totalDeposits
                    totalLiquidity
                    totalATokenSupply
                    reserveFactor
                    
                }
                }
            '''

In [22]:
V2_sample_transport       = RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
        verify=True,
        retries=5)

In [23]:
V2_client                 = Client(transport=V2_sample_transport)
V2_sample_response                  = V2_client.execute(gql(query))

In [24]:
V2_sample_response

{'reserves': [{'baseLTVasCollateral': '7000',
   'borrowingEnabled': True,
   'id': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c5990xb53c1a33016b2dc2ff3653530bff1848a515c8c5',
   'name': 'Wrapped BTC',
   'reserveFactor': '2000',
   'symbol': 'WBTC',
   'totalATokenSupply': '2885035776591',
   'totalDeposits': '2883273649494',
   'totalLiquidity': '2883404916961',
   'usageAsCollateralEnabled': True}]}

In [33]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
# requiredDate = "12-31-2021"

In [47]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    df['date']= pd.to_datetime(df['date'])
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    for required_row in df.iterrows():
        required_row = required_row[1]
        SUM_10 = []
        SUM_1 = []
            
        for row in aaveV2Res.iterrows():
            try:
                SUM_10.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(10)'])/100))
                SUM_1.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(1)'])/100))
            except:
                # print("ERROR")
                pass

        REQUIRED_1.append(sum(SUM_1))
        REQUIRED_10.append(sum(SUM_10))

        historicalVAR.loc[hist_ind] = [required_row['date']] + [sum(SUM_10)] + [sum(SUM_1)]
        hist_ind += 1 
        
    # Historical VAR for past 365 days    
    historicalVAR = historicalVAR[:365]
    
    REQUIRED_10 = historicalVAR.sort_values('VAR_10').iloc[3]
    REQUIRED_1 = historicalVAR.sort_values('VAR_1').iloc[3]

    REQUIRED_10_95 = historicalVAR.sort_values('VAR_10').iloc[9]
    REQUIRED_1_95 = historicalVAR.sort_values('VAR_1').iloc[9]

    var_date_10_99p = REQUIRED_10['date']
    var_date_10_95p = REQUIRED_10_95['date']
    var_date_1_99p = REQUIRED_1['date']
    var_date_1_95p = REQUIRED_1_95['date']
    
#     print(historicalVAR)
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS      = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P = str(REQUIRED_10['VAR_10'])
    VAR_DATE_10_DAY_99_P = var_date_10_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['10_day_95p'] = str(REQUIRED_10_95['VAR_10'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_10_day_95p'] = var_date_10_95p

#     # transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1[5])
#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_99p'] = var_date_1_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_95p'] = str(REQUIRED_1_95['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_95p'] = var_date_1_95p

    RELATIVE_VAR_ASSETS = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_ASSETS)
    RELATIVE_VAR_LIABILITIES = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [57]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate("01-08-2022")

/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [58]:
VAR_10_DAY_99_P

'-2.280979602053467'

In [59]:
TOTAL_ASSETS

5759136824.172598

In [60]:
TOTAL_LIABILITIES

13326307342.150661

In [61]:
priceChangeDf.head()

,timestamp,DAI_price,DAI_%change(10),GUSD_price,GUSD_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDP_price,...,AAVE_%change(1),AMPL_%change(1),BAT_%change(1),CRV_%change(1),DPI_%change(1),KNC_%change(1),MANA_%change(1),RENFIL_%change(1),ZRX_%change(1),date
5,1.641600e+09,1.003442,0.352049,1.005613,0.485078,1.002237,0.251932,1.002207,0.224243,1.002830,...,-4.32531,-3.29674,-8.56816,-8.28385,-5.85185,16.0999,-8.95729,-4.12405,-5.9064,2022-01-08
6,1.641514e+09,0.999931,0.12634,0.999058,0.0515365,1.000725,0.0972064,1.000598,0.0603426,1.003127,...,-3.64571,4.72254,-0.128368,-0.684873,-0.729374,3.2023,11.8041,-3.74759,-3.97712,2022-01-07
7,1.641427e+09,1.002725,0.210072,1.003478,0.139703,0.999175,-0.165428,1.001545,0.15005,0.994779,...,-8.04968,-12.989,-8.46815,-11.9537,-7.28864,-6.23575,-6.57908,-7.66419,-7.08895,2022-01-06
8,1.641341e+09,0.999962,0.0635983,0.998746,-0.0343808,1.002149,0.0471129,0.999808,-0.0186069,1.003751,...,-5.38003,1.77453,0.292024,-6.5058,-2.679,-3.37639,-1.8818,4.18632,-5.49026,2022-01-05
9,1.641254e+09,1.002154,0.0610655,1.000886,-0.243562,1.001556,-0.0199551,1.000829,-0.128579,1.002789,...,0.487017,-6.33703,-0.261016,9.15848,1.04085,-2.08402,-2.43436,0,6.28451,2022-01-04


In [31]:
# Test call for Aave V2 for current date
# This is also used for Position data for a particular date
# testRest = aaveV2HistoricalData(requiredDate)

In [32]:
# testRest

,Token,CurrentPrice,availableLiquidity,decimals,totalLiquidity,utilizationRate,Assets,Position,"Position($,Billion)"
0,TUSD,1.003795,4.909780e+07,18.0,1.165520e+08,0.609926,6.726787e+07,-4.928413e+07,-0.049284
1,RAI,3.049223,2.068256e+06,18.0,3.165761e+07,0.635126,2.535103e+07,-6.306572e+06,-0.006307
2,GUSD,1.000000,2.911591e+06,2.0,1.441706e+07,0.790126,1.150547e+07,-2.911591e+06,-0.002912
3,YFI,31531.350594,1.469237e+03,18.0,4.963135e+07,0.049524,3.304311e+06,-4.632704e+07,-0.046327
4,BAT,1.266438,4.199938e+06,18.0,5.827548e+06,0.086826,5.085856e+05,-5.318962e+06,-0.005319
5,MANA,3.347210,1.337645e+07,18.0,4.620238e+07,0.023812,1.428595e+06,-4.477378e+07,-0.044774
6,DPI,287.562513,1.789095e+05,18.0,5.172795e+07,0.005351,2.802739e+05,-5.144767e+07,-0.051448
7,UNI,17.074347,2.650525e+06,18.0,4.573173e+07,0.012909,4.757437e+05,-4.525598e+07,-0.045256
8,WBTC,46771.759300,2.647995e+04,8.0,1.267162e+09,0.023747,2.864824e+07,-1.238514e+09,-1.238514
9,REN,0.544649,4.418143e+07,18.0,2.465074e+07,0.021366,5.873646e+05,-2.406337e+07,-0.024063


[1548979200000, 3438.3604027386937]